# Renewable energy sources optimization

### Problem formulation

We need to optimize the distribution of the energy load from multiple sources to different consumers.
We need to to minimize the overall cost, while satisfying demand constraints.

### Parameters

- **Energy Sources**: Two sources (e.g., solar and wind), each with different capacities and costs.
- **Consumers**: Two consumers, each with a specific energy demand.
- **Objective**: Minimize the total cost of energy distribution while meeting all consumer demands without exceeding the source capacities.
- **Tools**: Linear programming, Quadratic Programming

- **Sources parameters**:

    | Source            | Capacity (kW) | Cost (€/kW) |
    |-------------------|---------------|-------------|
    | **I** (eg. Solar) | 100           | 0.15        |                    
    | **J**  (eg. Wind) | 150           | 0.20        |                   


- **Consumer Demands**:
    - Consumer A = 90 kW
    - Consumer B = 120 kW

### Formulation

- **Variables**:
    - $x_{i,a}$: Amount of energy transferred from source $i$ to consumer $a$.
- **Objective**
    We can define a linear cost, to be minimized:
    $$ \text{Cost} = cost_I \times (x_{I,A} + x_{I,B}) + cost_J \times(x_{J,A} + x_{J,B}) $$
- **Constraints**
  At any time these constraint must be met:
    1. Total energy supplied to each consumer meets their demand.
        - Demand of Consumer A: $x_{I,A} + x_{J,A} = 90$
        - Demand of Consumer B: $x_{I,B} + x_{J,B} = 120$
    2. The total energy taken from each source does not exceed its capacity.
        - Solar capacity: $x_{I,A} + x_{I,B} \leq 100$
        - Wind capacity: $x_{J,A} + x_{J,B} \leq 150$
    3. All energies must be positive
       - $x_{i,a} \geq 0$ for any $(i, a)$
         

In [36]:
from optimization.core.renewables import EnergySource, Consumer, EnergyDistribution

# Create EnergySource instances
solar = EnergySource("Solar", 30, 0.3)
wind = EnergySource("Wind", 30, 0.1)

# Create Consumer instances
consumer_a = Consumer("A", 20.)
consumer_b = Consumer("B", 10.)
consumer_c = Consumer("C", 9.)

# Create the energy distribution system
system = EnergyDistribution()

system.add_source(solar)
system.add_source(wind)

system.add_consumer(consumer_a)
system.add_consumer(consumer_b)
system.add_consumer(consumer_c)

In [37]:
system.consumers

{'A': {'demand': 20.0}, 'B': {'demand': 10.0}, 'C': {'demand': 9.0}}

In [38]:
system.sources

{'Solar': {'capacity': 30, 'cost_per_unit': 0.3, 'unit': 'kW'},
 'Wind': {'capacity': 30, 'cost_per_unit': 0.1, 'unit': 'kW'}}

In [39]:
import numpy as np
from scipy.optimize import minimize

x_array = np.array([
    [10., 20.], 
    [30., 40.],
    [50,  60.],
]) 

x_vector = x_array.flatten()
x_vector

array([10., 20., 30., 40., 50., 60.])

In [40]:
def objective(x):
    return solar.cost_per_unit * x[0::2].sum() + wind.cost_per_unit * x[1::2].sum()  # Using slicing to sum xs and ys, then xw and yw

assert system.calculate_cost(x_vector) == objective(x_vector)

system.calculate_cost(x_vector)

AssertionError: 

In [41]:
# Constraints
constraints = [
    {'type': 'ineq', 'fun': lambda x: solar.capacity - x[0::2].sum()}, # Sum of xs and ys should not exceed solar capacity
    {'type': 'ineq', 'fun': lambda x: wind.capacity - x[1::2].sum()},  # Sum of xw and yw should not exceed wind capacity
    {'type': 'eq', 'fun': lambda x: x[0:2].sum() - consumer_a.demand},   # Sum of xs and xw equals consumer_a's demand
    {'type': 'eq', 'fun': lambda x: x[2:4].sum() - consumer_b.demand},   # Sum of ys and yw equals consumer_b's demand
    {'type': 'eq', 'fun': lambda x: x[4:6].sum() - consumer_c.demand},   # Sum of ys and yw equals consumer_c's demand
]

assert minimize(system.calculate_cost,  x_vector, constraints=constraints).x.all() == minimize(objective,  x_vector, constraints=constraints).x.all()

AssertionError: 

In [42]:
# Solve
max_capacity = max( solar.capacity, wind.capacity )

solution = minimize(
    objective,  
    x_vector, 
    constraints=constraints,
    bounds = len(x_vector)*[(0,max_capacity)],
    method='SLSQP',
    options={'disp': True,'maxiter' : 100000}
)

solution

Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.699999999999999
            Iterations: 5
            Function evaluations: 35
            Gradient evaluations: 5


 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 5.699999999999999
       x: [ 3.167e+00  1.683e+01  3.167e+00  6.833e+00  2.667e+00
            6.333e+00]
     nit: 5
     jac: [ 3.000e-01  1.000e-01  3.000e-01  1.000e-01  3.000e-01
            1.000e-01]
    nfev: 35
    njev: 5

In [43]:
solution.x.reshape(3,2)

array([[ 3.16666667, 16.83333333],
       [ 3.16666667,  6.83333333],
       [ 2.66666667,  6.33333333]])

In [ ]:
import numpy as np
xrange = np.arange(0.4, max(consumer_a.demand, consumer_b.demand, consumer_c.demand), 0.4)
xrange = list(range(1,6))
xrange


In [ ]:
from itertools import product

def generate_matrix(xrange):    
    for indices in product(*[xrange] * 6):
        yield np.array(indices)

In [ ]:
xgrid = generate_matrix(xrange)

min_cost = 1e9

print(consumer_a.demand)
for x_vector in generate_matrix(xrange):
    
    if x_vector[0:2].sum() == consumer_a.demand and \
       x_vector[2:4].sum() == consumer_b.demand and \
       x_vector[4:6].sum() == consumer_c.demand and \
       x_vector[0::2].sum() <= solar.capacity and \
       x_vector[1::2].sum() <= wind.capacity:

        print(x_vector, x_vector[0::2].sum())
        
        cost = cost_function_vector(x_vector)
        
        if cost <= min_cost:
            min_cost = cost 
            print(x_vector, min_cost)

In [ ]:
from scipy.optimize import minimize

max_capacity = max( solar.capacity, wind.capacity )
x_guess = x_array.flatten()

print(x_vector)

res = minimize(
    #cost_function_vector,
    lambda x: solar.cost_per_unit * x_guess[0::2].sum() + wind.cost_per_unit * x_guess[1::2].sum(), #what we want to minimize
    x_guess, 

    bounds = ((0,max_capacity),(0,max_capacity),(0,max_capacity),(0,max_capacity),(0,max_capacity),(0,max_capacity)),
    method='SLSQP',options={'disp': True,'maxiter' : 100000}
)

res


import numpy as np
from scipy.optimize import minimize

# Objective function
def objective(x):
    xs, xw, ys, yw = x
    return a * (xs + ys) + b * (xw + yw)

# Constraints
constraints = [
    {'type': 'eq', 'fun': lambda x: x[0] + x[1] - Dx},  # xs + xw = Dx
    {'type': 'ineq', 'fun': lambda x: S - (x[0] + x[2])},  # xs + ys <= S
    {'type': 'eq', 'fun': lambda x: x[2] + x[3] - Dy},  # ys + yw = Dy
    {'type': 'ineq', 'fun': lambda x: W - (x[1] + x[3])}  # xw + yw <= W
]

# Coefficients
a = 0.3
b = 0.1
Dx = 13
S = 20
Dy = 10
W = 15

# Initial guesses
x0 = np.array([1, 1, 1, 1])

# Solve
solution = minimize(objective, x0, constraints=constraints)
print(solution)

In [ ]:
print(res)
print(res.fun)

### TensorFlow Implementation

We will use TensorFlow to approximate a solution via a gradient descent approach, treating constraints as penalty terms in the loss function.

In [ ]:
import tensorflow as tf

# Initialize variables
x_solar_A = tf.Variable(x_guess[0] , trainable=True)  
x_solar_B = tf.Variable(x_guess[2], trainable=True)
x_solar_C = tf.Variable(x_guess[4], trainable=True)
x_wind_A = tf.Variable(x_guess[1], trainable=True)
x_wind_B = tf.Variable(x_guess[3], trainable=True)
x_wind_C = tf.Variable(x_guess[5], trainable=True)

# Define constraints as penalties
def capacity_constraints():
    return [
        solar.capacity - (x_solar_A + x_solar_B + x_solar_C),  # Solar capacity
        wind.capacity - (x_wind_A + x_wind_B + x_wind_C),    # Wind capacity
        x_solar_A + x_wind_A - consumer_a.demand, # Demand of Consumer A
        x_solar_B + x_wind_B - consumer_b.demand, # Demand of Consumer B
        x_solar_C + x_wind_C - consumer_c.demand  # Demand of Consumer C
    ]

# Define sources, consumers, and their capacities and demands here

# Cost function
cost = solar.cost_per_unit * ( x_solar_A + x_solar_B + x_solar_C ) + wind.cost_per_unit * ( x_wind_A + x_wind_B + x_wind_C )

# Optimizer
optimizer = tf.optimizers.SGD(learning_rate=0.01)
optimizer = tf.optimizers.SGD(learning_rate=0.0001)  # Adjust learning rate here

# Training step
def train_step():
    with tf.GradientTape() as tape:
        constraints = capacity_constraints()
        penalty = tf.reduce_sum(tf.square(constraints))  # Squared penalties for constraint violation
        loss = cost + penalty
    gradients = tape.gradient(loss, [x_solar_A, x_solar_B, x_solar_C, x_wind_A, x_wind_B, x_wind_C])
    optimizer.apply_gradients(zip(gradients, [x_solar_A, x_solar_B, x_solar_C, x_wind_A, x_wind_B, x_wind_C]))
    return loss  # Return the loss value

# Optimization loop
for i in range(100):
    loss = train_step()
    if (i + 1) % 10 == 0:  # Print cost every 100 iterations
        print(f"Iteration {i + 1}, Cost: {loss.numpy()}")


In [ ]:
# Output the results
print(f'           |   Solar            |  Wind')
print(f'Consumer A |{x_solar_A.numpy()}  | {x_wind_A.numpy()}')
print(f'Consumer B |{x_solar_B.numpy()}  | {x_wind_B.numpy()}')
print(f'Consumer C |{x_solar_C.numpy()}  | {x_wind_C.numpy()}')


In [ ]:
loss.numpy()